# Document Similarity (vs.1) 

### First installing needed libraries

### Installing libraries related to OS

In [61]:

from os import listdir  # to list directories
from os.path import isfile, join  # to join files with path (regardless of what OS used)
import  os, docx2txt  # to convert word documents to text file/content


### Installing libraries related to languages

In [62]:
import nltk # Natural Language Processing Toolkit https://www.nltk.org/
import re  # regular expression 
from nltk.stem.isri import ISRIStemmer  # Arabic Stemmer
from nltk.util import ngrams # ngram for further read please follow the link 


### Installing libraries related to tabular and visualization

In [63]:
import pandas as pd
from IPython.display import display,HTML

### Cleaning function "Text"

In [64]:
# It receives string (containing text) and it returns two string, one is cleaned 
# the other cleaned and stemmed (Arabic) 

def basic_clean(text):
  """
  A simple function to clean up the data. All the words that
  are not designated as a stop word is then lemmatized after
  encoding and basic regex parsing are performed.
  """

  ADDITIONAL_STOPWORDS = []

  # reading arabic stopwords
  ADDITIONAL_STOPWORDS = []
  with open('./stopword_arabic.txt') as f:
      ADDITIONAL_STOPWORDS = [line.rstrip() for line in f ]

  wnl = nltk.stem.WordNetLemmatizer()
  stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS

  words = re.sub(r'[^\w\s]', '', text).split()
  st = ISRIStemmer()
  # other stemmer:
  #st = ARLSTem2()


  cleaned_txt = ' '.join([word for word in words if word not in stopwords])
  stemmed_txt =  ' '.join([st.stem(word) for word in words if word not in stopwords])
  return (cleaned_txt,stemmed_txt)


### Function that reads all file names in  a given folder

In [65]:
# It receive a folder path
# and it returns a list of files in that folder concatendated with absolute or relative path
def readFolderFiles(pathn):
    print('in readFolderFiles')
    print("Path = " + pathn)

    onlyfiles = [f for f in listdir(pathn) if isfile(join(pathn, f))]
    onlyfiles_fullpath = [join(pathn, f) for f in listdir(pathn) if isfile(join(pathn, f))]

    print(onlyfiles)
    print(onlyfiles_fullpath)
    return onlyfiles_fullpath

### Fundction that reads file content given file name

In [66]:
def readFileContent(file_path):
    print('In readFileContent for file  = ' + file_path )
    text = ''
    if file_path.endswith('.docx'):
        text = docx2txt.process(file_path)

    elif  file_path.endswith('.doc'):
        with open(file_path) as  f:
            text = f.read()
    return text


### Function that extract NGram (n=2 by default) from text

In [67]:

def gettingNgram(text ,ngram_n=2):

    ngrams_lst = []
    ngrams_lst_lst = []

    for line in text:
        token = nltk.word_tokenize(line)
        ngrams_lst_tmp = list(ngrams(token, ngram_n))
        ngrams_lst += ngrams_lst_tmp
        ngrams_lst_lst.append(ngrams_lst_tmp)
        #print(ngrams_lst)

    #print(bigrams_lst)
    return ngrams_lst,ngrams_lst_lst


### Main Operations

In [70]:
    # Use a breakpoint in the code line below to debug your script.
    pathn_lst = ['./docs/class1',
                 '/docs/class2',
                 ]
    class_lst_i = ['class1',
                   'class2'
                   ]

    lst_txt_cleaned_stemmed = []
    lst_txt_cleaned = []
    lst_txt_stemmed = []

    onlyfiles_fullpath=[]
    class_lst=[]
    for k,val in enumerate(pathn_lst):
        pathn = val
        classn = class_lst_i[k]
        files_lst = readFolderFiles(pathn)
        onlyfiles_fullpath += files_lst
        class_lst += [classn]*len(files_lst)

    print(class_lst)
    lst_filep_txt = []


    for i,v in enumerate(onlyfiles_fullpath):
        lst_filep_txt.append((v,readFileContent(v)))

    lst_txt =  [v[1] for v in lst_filep_txt]

    lst_txt_cleaned_stemmed += [basic_clean(v) for v in lst_txt]
    lst_txt_cleaned += [v[0] for v in lst_txt_cleaned_stemmed]
    lst_txt_stemmed += [v[1] for v in lst_txt_cleaned_stemmed]


    print('original  text')
    print(lst_txt)
    print('clean text')
    print(lst_txt_cleaned)
    print('stemmed  text')
    print(lst_txt_stemmed)
    print('bigram on original')
    bigram_lst,bigram_lst_lst  = gettingNgram(lst_txt)
    print(bigram_lst)
    print(bigram_lst_lst)
    bigram_lst_cleaned,bigram_lst_lst_cleaned = gettingNgram(lst_txt_cleaned)
    print(bigram_lst_cleaned)
    print(bigram_lst_lst_cleaned)
    bigram_lst_stemmed,bigram_lst_lst_stemmed = gettingNgram(lst_txt_stemmed)
    print(bigram_lst_stemmed)
    print(bigram_lst_lst_stemmed)
    df=[]

    data = {
        'File Absolute Path Name':onlyfiles_fullpath,
        'Class': class_lst,
        'File Text Content': lst_filep_txt,
         'File Text Cleaned': lst_txt_cleaned,
        'File Text Stemmed': lst_txt_stemmed,
        '(N=2) grams':bigram_lst_lst,
        '(N=2) grams-cleaned':bigram_lst_lst_cleaned,
        '(N=2) grams-stemmed': bigram_lst_lst_stemmed,
    }

    df = pd.DataFrame(data, columns=['File Absolute Path Name','Class','File Text Content', 'File Text Cleaned','File Text Stemmed',
                  '(N=2) grams','(N=2) grams-cleaned', '(N=2) grams-stemmed'])

    print(df)
    df.to_excel('./output/output_f.xls')

    


in readFolderFiles
Path = /Users/mac/PycharmProjects/TextAnalysis/docs/class1
['شرطة مكه.docx', 'منظمة الصحة والمتحور.docx']
['/Users/mac/PycharmProjects/TextAnalysis/docs/class1/شرطة مكه.docx', '/Users/mac/PycharmProjects/TextAnalysis/docs/class1/منظمة الصحة والمتحور.docx']
in readFolderFiles
Path = /Users/mac/PycharmProjects/TextAnalysis/docs/class2
['ودور الإيواء .docx']
['/Users/mac/PycharmProjects/TextAnalysis/docs/class2/ودور الإيواء .docx']
['class1', 'class1', 'class2']
In readFileContent for file  = /Users/mac/PycharmProjects/TextAnalysis/docs/class1/شرطة مكه.docx
In readFileContent for file  = /Users/mac/PycharmProjects/TextAnalysis/docs/class1/منظمة الصحة والمتحور.docx
In readFileContent for file  = /Users/mac/PycharmProjects/TextAnalysis/docs/class2/ودور الإيواء .docx
original  text
['وضح المتحدث الإعلامي لشرطة منطقة مكة المكرمة، أن دوريات أمن الطرق ضبطت مواطناً في العقد الثالث من العمر قام بنقل (9) مخالفين لنظام أمن الحدود من الجنسية اليمنية بمحافظة جدة - طريق الساحل، و

/var/folders/j4/zjyjq6lx02v0qr0jb_n217f00000gn/T/ipykernel_99400/2838115457.py:69: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel('/Users/mac/PycharmProjects/TextAnalysis/output/output_f.xls')


In [69]:
class_lst

['class1', 'class1', 'class2']

### Displaying Panda  Data

In [71]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199
df

,File Absolute Path Name,Class,File Text Content,File Text Cleaned,File Text Stemmed,(N=2) grams,(N=2) grams-cleaned,(N=2) grams-stemmed
0,/Users/mac/PycharmProjects/TextAnalysis/docs/class1/شرطة مكه.docx,class1,"(/Users/mac/PycharmProjects/TextAnalysis/docs/class1/شرطة مكه.docx, وضح المتحدث الإعلامي لشرطة منطقة مكة المكرمة، أن دوريات أمن الطرق ضبطت مواطناً في العقد الثالث من العمر قام بنقل (9) مخالفين لنظام أمن الحدود من الجنسية اليمنية بمحافظة جدة - طريق الساحل، وجرى إيقافهم واتخذت بحقهم الإجراءات النظامية الأولية، وإحالتهم إلى فرع النيابة العامة.)",وضح المتحدث الإعلامي لشرطة منطقة مكة المكرمة دوريات أمن الطرق ضبطت مواطنا العقد الثالث العمر بنقل 9 مخالفين لنظام أمن الحدود الجنسية اليمنية بمحافظة جدة طريق الساحل وجرى إيقافهم واتخذت بحقهم الإجراءات النظامية الأولية وإحالتهم فرع النيابة العامة,وضح تحدث علم شرط نطق مكة كرم دور امن طرق ضبط واط عقد ثلث عمر نقل 9 خلف نظم امن حدد جنس يمن حفظ جدة طرق سحل جرى يقف تخذ بحق جرء نظم ولي إحل فرع نيب عمة,"[(وضح, المتحدث), (المتحدث, الإعلامي), (الإعلامي, لشرطة), (لشرطة, منطقة), (منطقة, مكة), (مكة, المكرمة،), (المكرمة،, أن), (أن, دوريات), (دوريات, أمن), (أمن, الطرق), (الطرق, ضبطت), (ضبطت, مواطناً), (مواطناً, في), (في, العقد), (العقد, الثالث), (الثالث, من), (من, العمر), (العمر, قام), (قام, بنقل), (بنقل, (), ((, 9), (9, )), (), مخالفين), (مخالفين, لنظام), (لنظام, أمن), (أمن, الحدود), (الحدود, من), (من, الجنسية), (الجنسية, اليمنية), (اليمنية, بمحافظة), (بمحافظة, جدة), (جدة, -), (-, طريق), (طريق, الساحل،), (الساحل،, وجرى), (وجرى, إيقافهم), (إيقافهم, واتخذت), (واتخذت, بحقهم), (بحقهم, الإجراءات), (الإجراءات, النظامية), (النظامية, الأولية،), (الأولية،, وإحالتهم), (وإحالتهم, إلى), (إلى, فرع), (فرع, النيابة), (النيابة, العامة), (العامة, .)]","[(وضح, المتحدث), (المتحدث, الإعلامي), (الإعلامي, لشرطة), (لشرطة, منطقة), (منطقة, مكة), (مكة, المكرمة), (المكرمة, دوريات), (دوريات, أمن), (أمن, الطرق), (الطرق, ضبطت), (ضبطت, مواطنا), (مواطنا, العقد), (العقد, الثالث), (الثالث, العمر), (العمر, بنقل), (بنقل, 9), (9, مخالفين), (مخالفين, لنظام), (لنظام, أمن), (أمن, الحدود), (الحدود, الجنسية), (الجنسية, اليمنية), (اليمنية, بمحافظة), (بمحافظة, جدة), (جدة, طريق), (طريق, الساحل), (الساحل, وجرى), (وجرى, إيقافهم), (إيقافهم, واتخذت), (واتخذت, بحقهم), (بحقهم, الإجراءات), (الإجراءات, النظامية), (النظامية, الأولية), (الأولية, وإحالتهم), (وإحالتهم, فرع), (فرع, النيابة), (النيابة, العامة)]","[(وضح, تحدث), (تحدث, علم), (علم, شرط), (شرط, نطق), (نطق, مكة), (مكة, كرم), (كرم, دور), (دور, امن), (امن, طرق), (طرق, ضبط), (ضبط, واط), (واط, عقد), (عقد, ثلث), (ثلث, عمر), (عمر, نقل), (نقل, 9), (9, خلف), (خلف, نظم), (نظم, امن), (امن, حدد), (حدد, جنس), (جنس, يمن), (يمن, حفظ), (حفظ, جدة), (جدة, طرق), (طرق, سحل), (سحل, جرى), (جرى, يقف), (يقف, تخذ), (تخذ, بحق), (بحق, جرء), (جرء, نظم), (نظم, ولي), (ولي, إحل), (إحل, فرع), (فرع, نيب), (نيب, عمة)]"
1,/Users/mac/PycharmProjects/TextAnalysis/docs/class1/منظمة الصحة والمتحور.docx,class1,"(/Users/mac/PycharmProjects/TextAnalysis/docs/class1/منظمة الصحة والمتحور.docx, أعلنت منظمة الصحّة العالمية ليل الثلاثاء-الأربعاء أنّها تراقب نسخة متحوّرة جديدة من فيروس كورونا اسمها ""مو"" رُصدت للمرة الأولى في كولومبيا في يناير.\n\nوقالت المنظمة في نشرتها الوبائية الأسبوعية حول تطوّر الجائحة إنّ النسخة المتحوّرة بي.1.621 بحسب تسميتها العلمية - تمّ تصنيفها في الوقت الراهن ""متحوّرة يجب مراقبتها"".\n\nوأوضحت أنّ لدى هذه المتحوّرة طفرات يمكن أن تنطوي على خطر ""هروب مناعي"" (مقاومة للّقاحات)، الأمر الذي يجعل من الضروري إجراء مزيد من الدراسات عليها لفهم خصائصها بشكل أفضل.\n\nوجميع الفيروسات، بما في ذلك سارس-كوف-2 المسبّب لمرض كوفيد-19، تتحوّر بمرور الوقت، وإذا كان للغالبية العظمى من الطفرات تأثير ضئيل أو معدوم على خصائص الفيروس، إلا أنّ بعض هذه الطفرات يمكن أن يؤثّر على خصائص الفيروس كأن يزيد على سبيل المثال من سهولة انتشاره أو من مدى شدّة المرض الذي يسبّبه أو من مدى مقاومته للّقاحات أو الأدوية أو أدوات التشخيص أو غيرها من التدابير الاجتماعية والصحّية العامّة.\n\nوبسبب ظهور متحوّرات في نهاية 2020 شكّلت خطراً متزايداً على الصحّة العامّة عمدت منظمة الصحة العالمية إلى وضع قائمة بالمتحوّرات التي يجب مراقبتها وتلك المثيرة للقلق، 